In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [3]:
spark=SparkSession.builder.appName("week2").getOrCreate()

24/11/02 21:57:01 WARN Utils: Your hostname, Anants-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.68.3.53 instead (on interface en0)
24/11/02 21:57:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/02 21:57:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# spark.stop()

1) Implement a PySpark script that applies transformations like filter and withColumn on a DataFrame.


In [6]:
# # Sample data as a dictionary
# data_dict = {
#     "id": [1, 2, 3, 4],
#     "name": ["Alice", "Bob", "Cathy", "David"],
#     "age": [29, 32, 25, 35]
# }

# # Create a DataFrame from the dictionary
# df = spark.createDataFrame(data_dict)
csv_file_path = "week1.ipynb.csv"  # Replace with your CSV file path

# Read the CSV file into a DataFrame
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(csv_file_path)
print("Original DataFrame:")
df.show()

print("Filtered DataFrame:")
filtered_df=df.filter(col("marks")>95)
filtered_df.show()

Original DataFrame:
+----------+---------+--------------------+-----+--------------+
|first name|last name|             subject|marks|      location|
+----------+---------+--------------------+-----+--------------+
|     anant|  agarwal|               maths|   99|San Franscisco|
|   rishabh|    singh|                chem|   98|       Lucknow|
|    kanika|  agarwal|            commerce|   95|         dubai|
|   yashika|   mittal|           computers|   96|       seattle|
|   prasoon|   mittal|            commerce|   94|     bangalore|
| prashali |   mittal|Information techn...|   98|      Gurugram|
|  shivansh|bhatnagar|             biology|   94|       Lucknow|
+----------+---------+--------------------+-----+--------------+

Filtered DataFrame:
+----------+---------+--------------------+-----+--------------+
|first name|last name|             subject|marks|      location|
+----------+---------+--------------------+-----+--------------+
|     anant|  agarwal|               maths|   99|

In [7]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf,avg, sum ,explode, split, col, lower
def get_remark(marks):
    return "Good" if marks > 95 else "Improve"

get_remark_udf = udf(get_remark, StringType())

new_df=df.withColumn("Remark",get_remark_udf(df.marks))
new_df.show()


+----------+---------+--------------------+-----+--------------+-------+
|first name|last name|             subject|marks|      location| Remark|
+----------+---------+--------------------+-----+--------------+-------+
|     anant|  agarwal|               maths|   99|San Franscisco|   Good|
|   rishabh|    singh|                chem|   98|       Lucknow|   Good|
|    kanika|  agarwal|            commerce|   95|         dubai|Improve|
|   yashika|   mittal|           computers|   96|       seattle|   Good|
|   prasoon|   mittal|            commerce|   94|     bangalore|Improve|
| prashali |   mittal|Information techn...|   98|      Gurugram|   Good|
|  shivansh|bhatnagar|             biology|   94|       Lucknow|Improve|
+----------+---------+--------------------+-----+--------------+-------+



2) Write a PySpark script that performs actions like count and show on a DataFrame.

In [9]:
row_count=df.count()
print(f"Number of Rows are : {row_count}")

Number of Rows are : 7


In [10]:
first_three_rows = df.take(3)
print("First 3 rows:", first_three_rows)

First 3 rows: [Row(first name='anant', last name='agarwal', subject='maths', marks=99, location='San Franscisco'), Row(first name='rishabh', last name='singh', subject='chem', marks=98, location='Lucknow'), Row(first name='kanika', last name='agarwal', subject='commerce', marks=95, location='dubai')]


3) Demonstrate how to perform basic aggregations (e.g., sum, average) on a PySpark DataFrame.

In [12]:
total_marks=df.agg(sum("marks").alias("total marks")).collect()[0]["total marks"]
print(f"total marks :{total_marks}")

total marks :674


In [13]:
avg_marks=df.agg(avg("marks").alias("avg marks")).collect()[0]["avg marks"]
print(f"avg marks: {avg_marks}")

avg marks: 96.28571428571429


4) Show how to write a PySpark DataFrame to a CSV file.

In [15]:
output_path="marks.csv"
df.write.csv(output_path,header=True,mode="overwrite")

5) Implement wordcount program in PySpark.

In [17]:
data = [
    "Hello world",
    "Hello from PySpark",
    "Welcome to the world of PySpark",
    "Hello again world"
]
rdd = spark.sparkContext.parallelize(data)
word_counts=(rdd
            .flatMap(lambda line: line.split())
            .map(lambda word:(word.lower(),1))
            .reduceByKey(lambda a,b:a+b))
for word, count in word_counts.collect():
    print(f"{word}: {count}")

of: 1
pyspark: 2
the: 1
world: 3
to: 1
welcome: 1
hello: 3
from: 1
again: 1


In [18]:
data = [
    (1, "Hello world"),
    (2, "Hello from PySpark"),
    (3, "Welcome to the world of PySpark"),
    (4, "Hello again world")
]

df = spark.createDataFrame(data, ["id", "sentence"])
word_counts_df = (df
                  .select(explode(split(col("sentence"), " ")).alias("word"))  
                  .withColumn("word", lower(col("word")))
                  .groupBy("word")  
                  .count()
                  .orderBy("count", ascending=False))  

print("Word Counts:")
word_counts_df.show()

Word Counts:
+-------+-----+
|   word|count|
+-------+-----+
|  hello|    3|
|  world|    3|
|pyspark|    2|
|   from|    1|
|welcome|    1|
|    the|    1|
|     of|    1|
|     to|    1|
|  again|    1|
+-------+-----+



In [19]:
spark.stop()